In [1]:
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas

Sat Jun 13 2020 

CPython 3.8.3
IPython 7.13.0

numpy 1.18.1
scipy not installed
sklearn not installed
pandas 1.0.3

compiler   : Clang 10.0.0 
system     : Darwin
release    : 19.4.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit


In [2]:
import sys
import os
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
sys.path.append(os.path.join(PROJ_ROOT, "src"))

In [3]:
#from models.decision_tree import DecisionTree
import pandas as pd
import numpy as np

In [4]:
df1 = pd.read_excel(os.path.join(PROJ_ROOT, "data","processed","validate_data.xlsx"),index_col =0)
header = df1.columns.values.tolist()
labels = df1[["G3"]]

In [5]:
predictions = [13, 11, 12, 11, 13, 12, 10, 10, 12, 11, 16, 13, 9, 11, 12, 12, 13, 13, 12, 11, 13, 13, 11, 10, 11, 10, 10, 11, 12, 14, 13, 12, 13, 13, 12, 11, 13, 13, 13, 14, 10, 14, 14, 12, 14, 13, 17, 13, 10, 11, 13, 11, 16, 10, 12, 10, 11, 11, 13, 13, 10, 13, 13, 11, 13, 13, 16, 13, 13, 14, 12, 12, 11, 13, 12, 12, 10, 12, 10, 13, 10, 10, 11, 13, 11, 12, 10, 13, 13, 13, 13, 11, 10, 14, 10, 10, 11, 13, 10, 11, 13, 13, 12, 13, 14, 13, 16, 16, 13, 12, 15, 13, 12, 13, 10, 10, 10, 12, 14, 11, 13, 10, 11, 13, 11, 13, 10, 10, 13]
TSS = np.sum((np.array(labels) - np.array(labels).mean())** 2)
ESS = np.sum((np.array(predictions) - np.array(labels).mean()) ** 2)

ESS/TSS

0.49824720117607135

In [6]:
from models.utils import class_counts,is_numeric,partition,gini,gini_impurity,unique_vals
from models.decision_tree import DecisionTree,DecisionTree,Leaf,Question

ModuleNotFoundError: No module named 'utils'

In [28]:

  
    
class DecisionNode(DecisionTree):
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [239]:
import sys
sys.path.append('src')

import pandas as pd
import logging

logger = logging.getLogger(__name__)
GINI_IMPURITY = "gini"
INFO_GAIN = 'info_gain'

class DecisionTree: 
    
    def __init__(self,
                 question=None,
                 true_branch=None,
                 false_branch=None,
                 metrics=None, 
                 max_depth=None,
                 leafs=None,
                 final_leaf=None):
        
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.max_depth = max_depth
        self.final_leaf = final_leaf
        
        if metrics is None:
            self.metrics = GINI_IMPURITY     
   
        if leafs is None:
            self.leafs = []      
        
    def build_tree(self,train_data,header,count=0):
        
        """Builds the tree.
    
        Rules of recursion: 1) Believe that it works. 2) Start by checking
        for the base case (no further information gain). 3) Prepare for
        giant stack traces.
        """
        
        # Try partitioing the dataset on each of the unique attribute,
        # calculate the information gain,
        # and return the question that produces the highest gain.
        
        #print("n° features: {}".format(len(train_data[0]) - 1))
        gain, question = self.find_best_split(train_data,header)
        #print("--input data: {}".format(train_data))
        logger.info("--best question is ''{}'' with information gain: {}".format(question,round(gain,2)))
        # Base case: no further info gain
        # Since we can ask no further questions,
        # we'll return a leaf.
        
        print("--",count)
        print("--",gain)
        
        if isinstance(train_data,pd.DataFrame):
            train_data = train_data.values.tolist()
            
        if gain < 0.001:
            print("final_gain=",gain)
            
            leafs = self.leafs.append(Leaf(train_data))
            print(leafs)
            return DecisionTree(None,
                                self.true_branch, 
                                self.false_branch,
                                self.metrics,
                                self.max_depth,
                                leafs,
                                Leaf(train_data))
        
        if count == self.max_depth:
            print("final_depth=",count)
            self.leafs.append(Leaf(train_data))
            
            return DecisionTree(None,
                                self.true_branch, 
                                self.false_branch,
                                self.metrics,
                                self.max_depth,
                                self.leafs,
                                Leaf(train_data))
    
        true_rows, false_rows = partition(train_data, question)
        
        # Recursively build the true branch.
        logger.info("\n----TRUE BRANCH----")
        true_branch = self.build_tree(true_rows,header,count+1)
        
        # Recursively build the false branch.
        logger.info("\n----FALSE BRANCH----")
        false_branch = self.build_tree(false_rows,header,count+1)
        
        return DecisionTree(question,
                            true_branch, 
                            false_branch,
                            self.metrics,
                            self.max_depth,
                            self.leafs,
                            None)
    
    
    def initialize_split(self,train_data):
        
        if self.metrics == GINI_IMPURITY:
            # Calculate the information gain from this split
            current_uncertainty = gini(train_data)
               
        if self.metrics == INFO_GAIN:
            # TODO:
            pass
        
        return current_uncertainty
        
    def find_best_split(self,train_data,header):
        """Find the best question to ask by iterating over every feature / value
        and calculating the information gain."""
    
        if isinstance(train_data,pd.DataFrame):
            train_data = train_data.values.tolist()
        
        N_FEATURES = len(train_data[0]) - 1  # number of columns
        
        best_gain = 0  # keep track of the best information gain
        best_question = None  # keep train of the feature / value that produced it
        current_uncertainty = self.initialize_split(train_data)
        
        for col in range(N_FEATURES):  # for each feature
            
            values = unique_vals(train_data, col)  # unique values in the column
            logger.info("values features n° {} ''{}'': {}".format(col+1,header[col],values))
            for val in values:  # for each value
        
                question = Question(header,col, val)
                
                # try splitting the dataset
                true_rows, false_rows = partition(train_data, question)
                
                # Skip this split if it doesn't divide the dataset.
                if len(true_rows) == 1 or len(false_rows) == 1:
                    
                    continue
                
                if self.metrics == GINI_IMPURITY:
                    # Calculate the information gain from this split
                    gain = gini_impurity(true_rows, false_rows, current_uncertainty)
               
                if self.metrics == INFO_GAIN:
                    # TODO:
                    pass
               
                if gain >= best_gain:
                    best_gain, best_question = gain, question
    
        return best_gain, best_question


    def classify(self,row):
        """See the 'rules of recursion' above."""
        
        # Base case: we've reached a leaf
        if self.true_branch == None or self.true_branch == None:
            logger.info("predictions: {} ".format(self.final_leaf.predictions))
            return self.final_leaf.predictions
    
        print("tree_classifier->classify:",self.question)
        
        #print("input value {}".format(row[node.question.column]))
        
        #print(self.question.match(row))
        if self.question.match(row):
            print("yes",self)
            return self.true_branch.classify(row)
        else:
            print("no",self)
            return self.false_branch.classify(row)
    

    def __repr__(self, spacing=""):
        """World's most elegant tree printing function."""
    
        # Base case: we've reached a leaf
        if self.true_branch == None or self.true_branch == None:
            print (spacing + "Predict", str(print_leaf(self.final_leaf.predictions)))
            return "--- END ---"
        
        # Print the question at this node
        print (spacing + str(self.question))
    
        # Call this function recursively on the true branch
        print (spacing + '--> True:')
        self.true_branch.__repr__(spacing + "  ")
    
        # Call this function recursively on the false branch
        print (spacing + '--> False:')
        self.false_branch.__repr__(spacing + "  ")
        
        return "--- END ---"

In [240]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [241]:
a = {17: 3}
print_leaf(a)

{17: '100%'}

In [242]:
dt = DecisionTree()

In [243]:
dt = dt.build_tree(df1,df1.columns.values)

-- 0
-- 0.11716158855152903
-- 1
-- 0.1152524254401725
-- 2
-- 0.20132325141776936
-- 3
-- 0.2666666666666667
-- 4
-- 0.48
-- 5
-- 0.0
final_gain= 0.0
None
-- 5
-- 0.0
final_gain= 0.0
None
-- 4
-- 0.4444444444444445
-- 5
-- 0.0
final_gain= 0.0
None
-- 5
-- 0.0
final_gain= 0.0
None
-- 3
-- 0.22500000000000003
-- 4
-- 0.0
final_gain= 0.0
None
-- 4
-- 0.07999999999999993
-- 5
-- 0.0
final_gain= 0.0
None
-- 5
-- 0.0
final_gain= 0.0
None
-- 2
-- 0.10243803879310343
-- 3
-- 0.0
final_gain= 0.0
None
-- 3
-- 0.08866637461668436
-- 4
-- 0.1866666666666665
-- 5
-- 0.0
final_gain= 0.0
None
-- 5
-- 0.0
final_gain= 0.0
None
-- 4
-- 0.08801309493830281
-- 5
-- 0.2203856749311295
-- 6
-- 0.1111111111111111
-- 7
-- 0.5
-- 8
-- 0.0
final_gain= 0.0
None
-- 8
-- 0.0
final_gain= 0.0
None
-- 7
-- 0.0
final_gain= 0.0
None
-- 6
-- 0.0
final_gain= 0.0
None
-- 5
-- 0.31250000000000006
-- 6
-- 0.0
final_gain= 0.0
None
-- 6
-- 0.0
final_gain= 0.0
None
-- 1
-- 0.09510459280442024
-- 2
-- 0.04644322027753445
-- 3


In [244]:
dt.true_branch.true_branch.true_branch.true_branch

Is freetime >= 4?
--> True:
  Predict {17: '100%'}
--> False:
  Predict {18: '100%'}


--- END ---

In [246]:
dt.leafs

AttributeError: 'DecisionTree' object has no attribute 'leafs'

In [245]:
dt.classify(df1.values.tolist()[0])

tree_classifier->classify: Is G2 >= 12?
yes Is G2 >= 12?
--> True:
  Is G2 >= 14?
  --> True:
    Is G2 >= 15?
    --> True:
      Is G2 >= 17?
      --> True:
        Is freetime >= 4?
        --> True:
          Predict {17: '100%'}
        --> False:
          Predict {18: '100%'}
      --> False:
        Is activities == no?
        --> True:
          Predict {16: '100%'}
        --> False:
          Predict {15: '100%'}
    --> False:
      Is Fjob == at_home?
      --> True:
        Predict {15: '100%'}
      --> False:
        Is traveltime >= 2?
        --> True:
          Predict {15: '50%', 14: '50%'}
        --> False:
          Predict {14: '100%'}
  --> False:
    Is absences >= 12?
    --> True:
      Predict {14: '100%'}
    --> False:
      Is Fjob == services?
      --> True:
        Is traveltime >= 3?
        --> True:
          Predict {14: '66%', 13: '33%'}
        --> False:
          Predict {13: '100%'}
      --> False:
        Is Medu >= 3?
        --> True:
 

{14: 8}

In [ ]:
[
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]